<a href="https://colab.research.google.com/github/marcusflygar1-hash/AI_Course_Submissions_real/blob/main/Excersise_2_MarcusFlygar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
url = 'https://raw.githubusercontent.com/zhenliangma/Applied-AI-in-Transportation/master/Exercise_2_regression_model/Exercise2BusData.csv'
df = pd.read_csv(url)
df.head(10)

In [ ]:
df = df.drop(['Arrival_time', 'Stop_id','Bus_id','Line_id'], axis=1)
df.head(10)

In [ ]:
df.shape


In [ ]:
df.info()

In [ ]:
corr_matrix = df.corr() #Creates a correlation matrix
corr_matrix['Arrival_delay'].sort_values(ascending=False) #The correlation matrix focuses on the correlation between the remaining columns to Arrival Delay

As we can see the upstream stop delay is heavily correlated to the arrival delay, however this is not true for the remaining independent variables.

In [ ]:
x = df.drop('Arrival_delay', axis=1)
y = df['Arrival_delay']
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

The reason as to why we set the test_size = 0,2 is that this will devide the model into 80% training data and 20% of the data to test this on.

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)

Now we have trained and tested the data set, now we will evaluate the model and see if it performs well.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score
lin_mse = mean_squared_error(y_test, y_pred)
lin_mae = mean_absolute_error(y_test, y_pred)
lin_rmse = root_mean_squared_error(y_test, y_pred)
lin_r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {lin_mse}")
print(f"Mean Absolute Error: {lin_mae}")
print(f"Root Mean Squared Error: {lin_rmse}")
print(f"R2 Score: {lin_r2}")

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)  # Plot actual vs. predicted values

# Add labels and title
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs. Predicted Values")

# Add a diagonal line for reference (perfect predictions)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red', lw=2)

# Show the plot
plt.show()

Now that we have performed a Linear regression together with training and test data and evaluated it. It is time to perform a linear regression witht the XgBoost package.  

In [ ]:
from xgboost import XGBRegressor
xgb_reg = XGBRegressor()
xgb_reg.fit(X_train, y_train)

In [ ]:
#Setting upp the features and labels into a Dmatrix so XGBoost can read it
#(Maybe idk, but I read that you should do this)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
# Setting the booster parameters, these are just generic, I might play around
# with them later and if needed
param_xg = {'max_depth':5,
            'eta':0.1,
            'objective': 'reg:squarederror',
            'seed': 42}
###
param_xg['nthread'] = 4
######
param_xg['eval_metric'] = 'auc'


In [ ]:
evallist = [(dtrain, 'train'), (dtest, 'eval')]

In [ ]:
#Here I set upp the training and saving the model.
# This so I can evaluate it later
num_round = 10
bst = xgb.train(param_xg, dtrain, num_round, evallist)
bst.save_model('xgboost_model.model')

In [ ]:
dtest = xgb.DMatrix(X_test)
y_pred_xgb = bst.predict(dtest)